**Rust Quantum Library**

Below are dependencies, creates, and global variables.

In [2]:
:dep ndarray = { version = "0.14", features = ["serde"] }
:dep ndarray-linalg = { version = "0.13.1", features = ["intel-mkl-static"]}
:dep plotters = { git = "https://github.com/38/plotters", default_features = false, features = ["evcxr", "all_series"] }
:dep thiserror = "1.0"
:dep serde = { version = "1.0", features = ["derive"] }
:dep serde_json = "1.0"
:dep dimensioned = "0.7"
:dep num-complex = "0.4"
:dep qm = { version = "0.1", git ="https://github.com/buff-beacon-project/qm" }
:dep ndarray-csv = "0.5.1"
extern crate plotters;
extern crate ndarray;

pub use ndarray_linalg::c64;
use std::f64::consts::PI;
use plotters::prelude::*;
use ndarray::prelude::*;
use ndarray_linalg::*;
use qm::*;

extern crate csv;
extern crate ndarray_csv;
use csv::{ReaderBuilder};
use ndarray_csv::{Array2Reader};
use std::error::Error;
use std::fs::File;

pub type VecC64 = ndarray::Array1<c64>;
pub type MatrixC64 = ndarray::Array2<c64>;

In [ ]:
pub fn read_c64_array(csv_file: String, array_size: usize) -> Result<MatrixC64, Box<dyn Error>> {

    let file = File::open(csv_file)?;
    let mut reader = ReaderBuilder::new().has_headers(false).from_reader(file);
    let array_read: MatrixC64 = reader.deserialize_array2((array_size, array_size))?;
    Ok(array_read)
}

pub fn read_f64_vector(csv_file: String) -> Result<VecF64, Box<dyn Error>> {

    let file = File::open(csv_file)?;
    let mut reader = ReaderBuilder::new()
        .from_reader(file);

    let v : Vec<f64> = reader.headers()?
        .into_iter()
        .map(|s| s.parse::<f64>().unwrap()) // to f64
        .collect(); // create a vec      
    
    let arr = Array::from(v);
    Ok(arr)
}

let signal: VecF64 = array![1.4446824684143E-06,1.49734123420715E-06,1.54999999999999E-06,1.60265876579284E-06,1.65531753158569E-06];
let idler: VecF64 = array![1.45728222027807E-06,1.51093173528974E-06,1.56458125030141E-06,1.61823076531308E-06,1.67188028032476E-06];
let jsa: MatrixC64 = read_c64_array("jsa_data5.csv".to_string(),5).unwrap();

let norm = two_source_hom_norm(signal.clone(), idler.clone(), jsa.clone());
println!("norm value is {}", norm);
let times = Array::linspace(-300.0e-15_f64, 300.0e-15_f64, 100);
let mut points_time: Vec<f64> = Vec::new();
let mut points_ss: Vec<f64> = Vec::new();
let mut points_ii: Vec<f64> = Vec::new();

for i in 0..99{
    let rate = find_two_source_hom(signal.clone(), idler.clone(), jsa.clone(), times[i]);
    let rate_ss = rate.0/norm;
    let rate_ii = rate.1/norm;
    points_time.push(times[i]*1.0e+15_f64);
    points_ss.push(rate_ss);
    points_ii.push(rate_ii);
}
println!("ss is {:?}", points_ss);
println!("ii is {:?}", points_ii);

let figure = evcxr_figure((600, 400), |root| {
    root.fill(&WHITE);
    let mut chart = ChartBuilder::on(&root)
        .caption("Huong-Ou-Mandel graph for four photons", ("Arial", 20).into_font())
        .margin(5)
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(-350_f64..350_f64, 0_f64..1_f64)?;
    
    chart
        .configure_mesh()
        .x_desc("Time Delay (fs)")
        .y_desc("Coincidence Probability")
        .draw()?;
    
    chart
        .draw_series(LineSeries::new(
            points_time.clone().into_iter().zip(points_ss.into_iter()), &RED,))?
        .label("signal-signal")
        .legend(|(x, y)| PathElement::new(vec![(x, y), (x + 20, y)], &RED));        
    
    chart.draw_series(LineSeries::new(
        points_time.clone().into_iter().zip(points_ii.into_iter()), &BLUE,))?
        .label("idler-idler")
        .legend(|(x, y)| PathElement::new(vec![(x, y), (x + 20, y)], &BLUE));    

    chart
    .configure_series_labels()
    .background_style(&WHITE.mix(0.8))
    .border_style(&BLACK)
    .draw()?;
    Ok(())
});
figure